[AZURE ML PRODUCTION DEPLOYMENT - FOQA DECISION TREE MODEL](https://c3.ndc.nasa.gov/dashlink/resources/1018/)

# 1. Packages installation - AML

In [38]:
!which python

/anaconda/envs/azureml_py38/bin//python


In [ ]:
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall scikit-learn
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall -U imbalanced-learn
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall azureml
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall azure-ai-ml
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall azureml-inference-server-http
!/anaconda/envs/azureml_py38/bin/python -m pip install --upgrade --force-reinstall azure-ai-formrecognizer

# 2. Init Azure Config

In [10]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = '<insert-subscription-id>'
resource_group = '<insert-resource-group>'
workspace_name = '<insert-workspace-name>'

# Get a handle to the workspace
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

Explore Azure Dataset

In [ ]:
'''
from azureml.core import Workspace, Dataset

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='foqa-data-asset')
dataset.download(target_path='.', overwrite=True)
# Download mounts the file as local file
# TODO: How to work directly without mounting

import numpy as np
import os
with dataset.mount() as mount_context:
    # print(os.listdir(mount_context.mount_point)[0])
    full_data = np.load(os.listdir(mount_context.mount_point)[0])

data = full_data['data']
label = full_data['label']
print("Data:",data.shape)
print("Label:",label.shape)'''

# 3. Create training script

In [12]:
import os
train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

In [13]:
%%writefile {train_src_dir}/main.py
import numpy as np

def main():
    """Main function of the script."""
    pass

if __name__ == "__main__":
    main()    

Overwriting ./src/main.py


# 4. Create custom environment

In [14]:
%%writefile ./conda.yaml
name: foqa-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib
  - xgboost
  - imbalanced-learn  
  - pip
  - pip:
    - azureml
    - azure-ai-ml
    - azureml-mlflow
    - azureml-inference-server-http

Overwriting ./conda.yaml


In [15]:
from azure.ai.ml.entities import Environment

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./conda.yaml",
    name="foqa-env",
    description="Environment for FOQA",
)
ml_client.environments.create_or_update(env)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'foqa-env', 'description': 'Environment for FOQA', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/e3fb51e5-d8bd-4bf8-9685-bda3d5d2e216/resourceGroups/foqa-resource-2/providers/Microsoft.MachineLearningServices/workspaces/foqa-ws-2/environments/foqa-env/versions/3', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/foqa-compute/code/Users/duc.tran', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f5574109d30>, 'serialize': <msrest.serialization.Serializer object at 0x7f5574109e80>, 'version': '3', 'latest_version': None, 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.7', 'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'xgboost', 'imbalanced-learn', 'pip', {'pip': ['azureml', 'azure-ai-ml', 'azureml-mlflow', 'azureml-inference-server-http']}], 'name': 'foqa

# 5. Create compute cluster